In [1]:
import numpy as np
import pandas as pd

In [2]:
from label_encoder import LabelEncode
from kfold import KFold, StratifiedKFold
from metrics import accuracy_score, classification_report, confusion_matrix, f1_score

In [3]:
data = pd.read_csv('GLASS.csv', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [4]:
print('Total no of examples ::', len(data))

Total no of examples :: 214


In [5]:
print('Is any data missing ::', data.isnull().values.any())

Is any data missing :: False


In [6]:
data[10].value_counts()

2    76
1    70
7    29
3    17
5    13
6     9
Name: 10, dtype: int64

### Only six classes

In [7]:
lb = LabelEncode()
data[10] = lb.fit_transform(data[10])

In [8]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,0
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,0
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,0
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,0
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,0


In [9]:
data[10].value_counts()

1    76
0    70
5    29
2    17
3    13
4     9
Name: 10, dtype: int64

In [10]:
from sklearn.svm import SVC

/home/akash/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88



In [12]:
X, y = data.drop(columns = [0, 10]).values, data[10].values

In [13]:
for train_index, test_index in StratifiedKFold(n_splits=10).split(X, y):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
#     print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    model = SVC(C=0.1, kernel='poly', gamma='auto')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
#     print(y_pred.shape, y_train_pred.shape)
    print('Train Accuracy :: ', accuracy_score(y_train, y_train_pred))
#     print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Train Accuracy ::  0.8072916666666666
              precision    recall  f1-score   support

           0       0.60      0.43      0.50         7
           1       0.58      0.88      0.70         8
           2       1.00      0.50      0.67         2
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         3

    accuracy                           0.68        22
   macro avg       0.70      0.63      0.64        22
weighted avg       0.68      0.68      0.66        22



SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Train Accuracy ::  0.8229166666666666
              precision    recall  f1-score   support

           0       0.45      0.71      0.56         7
           1       0.60      0.38      0.46         8
           2       0.00      0.00      0.00         2
           3       1.00      1.00      1.00         1
           4       0.50      1.00      0.67         1
           5       1.00      0.67      0.80         3

    accuracy                           0.55        22
   macro avg       0.59      0.63      0.58        22
weighted avg       0.57      0.55      0.53        22



SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

...

Train Accuracy ::  0.8031088082901554
              precision    recall  f1-score   support

           0       0.43      0.43      0.43         7
           1       0.83      0.71      0.77         7
           2       0.00      0.00      0.00         2
           3       0.50      1.00      0.67         1
           4       0.00      0.00      0.00         1
           5       0.75      1.00      0.86         3

    accuracy                           0.57        21
   macro avg       0.42      0.52      0.45        21
weighted avg       0.55      0.57      0.55        21



SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Train Accuracy ::  0.7823834196891192
              precision    recall  f1-score   support

           0       0.70      1.00      0.82         7
           1       0.83      0.71      0.77         7
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       0.75      1.00      0.86         3

    accuracy                           0.76        21
   macro avg       0.55      0.62      0.57        21
weighted avg       0.67      0.76      0.70        21



SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Train Accuracy ::  0.7927461139896373
              precision    recall  f1-score   support

           0       0.60      0.43      0.50         7
           1       0.60      0.86      0.71         7
           2       0.00      0.00      0.00         2
           3       0.50      1.00      0.67         1
           4       1.00      1.00      1.00         1
           5       1.00      0.67      0.80         3

    accuracy                           0.62        21
   macro avg       0.62      0.66      0.61        21
weighted avg       0.61      0.62      0.60        21

